In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_chroma import Chroma


In [2]:
os.environ['OPENAI_API_KEY']=os.environ.get('OPENAI_API_KEY')

In [3]:
model = ChatOpenAI()

In [8]:
# Load CSV files
csv_loader = DirectoryLoader('dataset', glob="**/*.csv")
pdf_loader1 = PyPDFLoader('dataset/CBSE_MH_Manual.pdf')
pdf_loader2 = PyPDFLoader('dataset/MentalHealthWorkbook_7Books.pdf')
pdf_loader3 = PyPDFLoader('dataset/MentalWellnessAHolisticApproachToMentalHealthAndHealing.pdf')

all_documents = []
all_documents.extend(csv_loader.load())
all_documents.extend(pdf_loader1.load())
all_documents.extend(pdf_loader2.load())
all_documents.extend(pdf_loader3.load())

In [9]:
# splitting the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=300)
texts = text_splitter.split_documents(all_documents)

In [10]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(texts, embedding_function)

/home/employee/Desktop/Hachathon/mha-llm/hackvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/employee/Desktop/Hachathon/mha-llm/hackvenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/employee/Desktop/Hachathon/mha-llm/hackvenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be re

In [27]:
# query it
query = "Is yoga good for mental health"


In [18]:
# docs = db.similarity_search(query)
# print(docs[0].page_content)

Learn to set healthy boundaries: Talk to your employer about your concerns and try to establish clear expectations around your workload and compensation. If necessary, seek support from a mentor or professional advisor.\n4. Cultivate positive thinking: Try to focus on the things that you are grateful for and the control that you do have in your life. Engage in positive self-talk and surround yourself with supportive people.\n5. Use problem-solving skills: Identify the root cause of your stressors, and brainstorm possible solutions. This can help you feel more in control and empowered in the situation.\nRemember, managing stress is an ongoing process that requires consistent effort and self-care. It's important to stay committed to your stress management strategies, even in challenging times.


In [19]:
# save to disk
db2 = Chroma.from_documents(texts, embedding_function, persist_directory="./chroma_db")

In [37]:

docs = db2.similarity_search(query)
print(docs[0].page_content)

body awareness, and an enhanced sense of vitality. Often, 
practitioners experience a reduction in insomnia and more 
restful sleep. Yoga has also been shown to increase the level 
of gamma-aminobutyric acid (GABA), a chemical in the brain 
that helps to regulate nerve activity. This is especially helpful 
for anxiety sufferers, who can have low levels of GABA. There 
is also evidence that yoga can help to reduce symptoms of 
anxiety and low mood in those suffering from depression. 
And, with practice, the process of meditation (see p.206), in 
which we continually let go of any thoughts that arrive, 
brings us closer to feeling centered and peaceful.
“Yoga  practice  promotes  
r ela xa tion, a mor e po si tive 
outlook, and an enhanced 
se n se o f vi tali ty. ”
US_184-185_Yoga.indd   184US_184-185_Yoga.indd   184 30/11/2020   11:50am30/11/2020   11:50am


In [22]:
# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

In [28]:
docs = db3.similarity_search(query)
print(docs[0].page_content)

body awareness, and an enhanced sense of vitality. Often, 
practitioners experience a reduction in insomnia and more 
restful sleep. Yoga has also been shown to increase the level 
of gamma-aminobutyric acid (GABA), a chemical in the brain 
that helps to regulate nerve activity. This is especially helpful 
for anxiety sufferers, who can have low levels of GABA. There 
is also evidence that yoga can help to reduce symptoms of 
anxiety and low mood in those suffering from depression. 
And, with practice, the process of meditation (see p.206), in 
which we continually let go of any thoughts that arrive, 
brings us closer to feeling centered and peaceful.
“Yoga  practice  promotes  
r ela xa tion, a mor e po si tive 
outlook, and an enhanced 
se n se o f vi tali ty. ”
US_184-185_Yoga.indd   184US_184-185_Yoga.indd   184 30/11/2020   11:50am30/11/2020   11:50am


In [36]:
template = """
The following tool combines the power of AI-powered conversation with creative expression tools to empower individuals in their journey toward mental well-being. It's crucial to remember that this application should never be a substitute for professional medical advice or therapy.

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.


{context}

Question: {question}
Helpful Answer:
"""

prompt_template = PromptTemplate(
    input_variables=['text'], 
    template=template
)

In [42]:
retriever = db3.as_retriever(search_type="mmr",
                          search_kwargs={'k': 5, 'fetch_k': 20})

qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(temperature=0,
               max_tokens=512),
    retriever=retriever,
    return_source_documents=True,
    return_generated_question=True,
    
)

qa.combine_docs_chain.llm_chain.prompt = prompt_template
chat_history = []

In [39]:
print(qa)

combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nThe following tool combines the power of AI-powered conversation with creative expression tools to empower individuals in their journey toward mental well-being. It's crucial to remember that this application should never be a substitute for professional medical advice or therapy.\n\nUse the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n\n{context}\n\nQuestion: {question}\nHelpful Answer:\n"), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x7d96a5df5810>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7d9616cb2b30>, model_name='gpt-3.5-turbo', temperature=0.0, max_tokens=512, openai_api_key=SecretStr('**********'), openai_proxy='')), document_variable_name='context') question_generator=LLMChain(pr

In [43]:
question = "Which yoga is better for mental health?"
result = qa({"question": question, "chat_history": chat_history})
chat_history.extend([(question, result["answer"])])
print(result['answer'])


There is no one "better" type of yoga for mental health, as different types of yoga can offer different benefits. However, Hatha yoga is often recommended for its focus on relaxation and flexibility, while Ashtanga yoga can be beneficial for building strength and endurance. Ultimately, the best type of yoga for mental health will depend on an individual's specific needs and preferences.


In [44]:
print(chat_history)

[('Which yoga is better for mental health?', '\nThere is no one "better" type of yoga for mental health, as different types of yoga can offer different benefits. However, Hatha yoga is often recommended for its focus on relaxation and flexibility, while Ashtanga yoga can be beneficial for building strength and endurance. Ultimately, the best type of yoga for mental health will depend on an individual\'s specific needs and preferences.')]


In [45]:
question = "How often do we need to do it?"
result = qa({"question": question, "chat_history": chat_history})
chat_history.extend([(question, result["answer"])])
print(result['answer'])



There is no set frequency for practicing yoga for mental health as it can vary for each individual. Some people may benefit from practicing daily, while others may find a few times a week to be sufficient. It's important to listen to your body and find a routine that works best for you. It's also important to consult with a professional if you have any underlying health conditions before starting a new exercise routine.


In [46]:
print(chat_history)

[('Which yoga is better for mental health?', '\nThere is no one "better" type of yoga for mental health, as different types of yoga can offer different benefits. However, Hatha yoga is often recommended for its focus on relaxation and flexibility, while Ashtanga yoga can be beneficial for building strength and endurance. Ultimately, the best type of yoga for mental health will depend on an individual\'s specific needs and preferences.'), ('How often do we need to do it?', "\nThere is no set frequency for practicing yoga for mental health as it can vary for each individual. Some people may benefit from practicing daily, while others may find a few times a week to be sufficient. It's important to listen to your body and find a routine that works best for you. It's also important to consult with a professional if you have any underlying health conditions before starting a new exercise routine.")]


In [47]:
question = "I am a 24 year old male, now tell me"
result = qa({"question": question, "chat_history": chat_history})
chat_history.extend([(question, result["answer"])])
print(result['answer'])

As an AI, I am not qualified to give medical advice. It is important to consult with a healthcare professional before starting any new exercise regimen. However, studies have shown that regular yoga practice can have positive effects on mental health, including reducing stress and improving mood. It is recommended to aim for about 150 minutes of moderate- to high-intensity activity per week, which can be achieved through 30 minutes of yoga practice five times a week. However, the frequency and intensity of yoga practice may vary for each individual, so it is important to listen to your body and adjust accordingly.


In [48]:
print(chat_history)

[('Which yoga is better for mental health?', '\nThere is no one "better" type of yoga for mental health, as different types of yoga can offer different benefits. However, Hatha yoga is often recommended for its focus on relaxation and flexibility, while Ashtanga yoga can be beneficial for building strength and endurance. Ultimately, the best type of yoga for mental health will depend on an individual\'s specific needs and preferences.'), ('How often do we need to do it?', "\nThere is no set frequency for practicing yoga for mental health as it can vary for each individual. Some people may benefit from practicing daily, while others may find a few times a week to be sufficient. It's important to listen to your body and find a routine that works best for you. It's also important to consult with a professional if you have any underlying health conditions before starting a new exercise routine."), ('I am a 24 year old male, now tell me', 'As an AI, I am not qualified to give medical advice